# Laboratorio 1: Análisis de Datos E-commerce (SOLUCIÓN)

Este notebook contiene la solución completa del laboratorio de análisis de datos de e-commerce.

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Configurar visualización
plt.style.use('Solarize_Light2')
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 6]
sns.set_palette('husl')

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print('Ambiente configurado correctamente')

## 1. Exploración Inicial

Primero cargaremos y exploraremos los datos de ventas.

In [ ]:
# Crear dataset de ejemplo
np.random.seed(42)
data = {
    'fecha': pd.date_range('2024-01-01', periods=1000, freq='H'),
    'producto': np.random.choice(['Laptop', 'Smartphone', 'Tablet', 'Auriculares'], 1000),
    'categoria': np.random.choice(['Electrónica', 'Accesorios'], 1000),
    'precio': np.random.uniform(100, 1000, 1000).round(2),
    'cantidad': np.random.randint(1, 5, 1000),
    'canal': np.random.choice(['Online', 'Tienda'], 1000)
}

df = pd.DataFrame(data)

# Exploración inicial
print('Primeras 5 filas del DataFrame:')
display(df.head())

print('\nInformación del DataFrame:')
df.info()

print('\nEstadísticas descriptivas:')
display(df.describe())

print('\nValores únicos por columna:')
for col in ['producto', 'categoria', 'canal']:
    print(f'\n{col.title()}:')
    display(df[col].value_counts())

## 2. Análisis de Ventas

Calculamos métricas de ventas y creamos visualizaciones básicas.

In [ ]:
# Calcular ventas
df['ventas'] = df['precio'] * df['cantidad']

# Métricas generales
metricas = {
    'Ventas Totales': df['ventas'].sum(),
    'Ticket Promedio': df['ventas'].mean(),
    'Número de Transacciones': len(df),
    'Cantidad Total Vendida': df['cantidad'].sum()
}

print('Métricas Generales:')
metricas_df = pd.DataFrame(metricas.items(), columns=['Métrica', 'Valor'])
metricas_df['Valor'] = metricas_df['Valor'].round(2)
display(metricas_df)

# Análisis por categoría
print('\nVentas por Categoría:')
ventas_categoria = df.groupby('categoria').agg({
    'ventas': ['sum', 'mean', 'count'],
    'cantidad': 'sum'
}).round(2)
display(ventas_categoria)

# Top productos
print('\nTop Productos por Ventas:')
display(df.groupby('producto')['ventas'].sum().sort_values(ascending=False))

In [ ]:
# Visualizaciones de ventas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Ventas por categoría
sns.barplot(data=df, x='categoria', y='ventas', estimator='sum', ax=axes[0,0])
axes[0,0].set_title('Ventas Totales por Categoría')
axes[0,0].set_ylabel('Ventas ($)')

# 2. Distribución por producto
df.groupby('producto')['ventas'].sum().plot(kind='pie', autopct='%1.1f%%', ax=axes[0,1])
axes[0,1].set_title('Distribución de Ventas por Producto')

# 3. Histograma de ventas
sns.histplot(data=df, x='ventas', bins=30, ax=axes[1,0])
axes[1,0].set_title('Distribución de Montos de Venta')
axes[1,0].set_xlabel('Monto de Venta ($)')

# 4. Box plot de precios
sns.boxplot(data=df, x='categoria', y='precio', ax=axes[1,1])
axes[1,1].set_title('Distribución de Precios por Categoría')
axes[1,1].set_ylabel('Precio ($)')

plt.tight_layout()
plt.show()

## 3. Análisis Temporal

Analizamos patrones y tendencias en el tiempo.

In [ ]:
# Análisis temporal
df['hora'] = df['fecha'].dt.hour
df['dia'] = df['fecha'].dt.date

# Ventas por día
ventas_diarias = df.groupby('dia')['ventas'].sum()

# Calcular tendencia
tendencia = ventas_diarias.rolling(window=24).mean()

# Visualizar tendencia
plt.figure(figsize=(15, 6))
plt.plot(ventas_diarias.index, ventas_diarias.values, label='Ventas diarias', alpha=0.5)
plt.plot(tendencia.index, tendencia.values, label='Tendencia (24 horas)', linewidth=2)
plt.axhline(y=ventas_diarias.mean(), color='r', linestyle='--', label='Promedio')

plt.title('Tendencia de Ventas')
plt.xlabel('Fecha')
plt.ylabel('Ventas ($)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis por hora y canal
# Ventas por hora
ventas_hora = df.groupby('hora')['ventas'].agg(['mean', 'sum', 'count']).round(2)
print('Análisis de ventas por hora:')
display(ventas_hora)

# Ventas por canal
ventas_canal = df.groupby(['canal', 'producto'])['ventas'].sum().unstack()
print('\nVentas por canal y producto:')
display(ventas_canal)

# Visualizar distribución por canal
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='canal', y='ventas')
plt.title('Distribución de Ventas por Canal')
plt.show()

## 4. Análisis de Correlaciones

In [ ]:
# Matriz de correlación
variables_numericas = ['precio', 'cantidad', 'ventas', 'hora']
correlaciones = df[variables_numericas].corr()

# Visualizar correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()

# Correlaciones más importantes
print('\nCorrelaciones más significativas:')
corr_flat = correlaciones.unstack()
corr_ord = corr_flat[corr_flat != 1.0].abs().sort_values(ascending=False)
display(corr_ord.head())

## 5. Conclusiones

### Principales Hallazgos:

1. **Métricas Clave de Ventas**
   - Se analizaron {len(df)} transacciones
   - Ventas totales: ${df['ventas'].sum():,.2f}
   - Ticket promedio: ${df['ventas'].mean():,.2f}

2. **Patrones por Categoría**
   - La categoría más vendida fue {df.groupby('categoria')['ventas'].sum().idxmax()}
   - El producto más vendido fue {df.groupby('producto')['ventas'].sum().idxmax()}

3. **Patrones Temporales**
   - Se identificaron picos de venta en determinadas horas
   - La tendencia general muestra {tendencia.diff().mean() > 0 and 'crecimiento' or 'decrecimiento'}

4. **Patrones por Canal**
   - El canal dominante es {df.groupby('canal')['ventas'].sum().idxmax()}
   - Existen diferencias significativas entre canales

### Recomendaciones:

1. Optimizar el inventario basado en patrones de venta
2. Desarrollar estrategias específicas por canal
3. Aprovechar las horas pico identificadas
4. Implementar promociones basadas en datos